In [65]:

from pyspark.sql.functions import round
from lib.common_functions import *
from lib.configuration import *

In [ ]:
spark = get_spark_session('ml-100k')
spark.active()

## Friends Dataset

In [ ]:
df_data = spark.read.format("csv").option("header", "false").option("inferSchema", "true").load(f'{ml_100k_input_path}/friends.csv', delimiter=',')

df_friends = df_data.withColumnRenamed('_c0', 'id').withColumnRenamed('_c1', 'name').withColumnRenamed('_c2', 'age').withColumnRenamed('_c3', 'num_friends')

df_friends.show()
#df_friends.count() 

In [ ]:
df_friends.printSchema()

In [ ]:
df_friends.createOrReplaceTempView("friends")
teenagers = spark.sql("SELECT * FROM friends WHERE age >= 13 AND age <= 19 ORDER BY 4 DESC")
teenagers.show()

In [ ]:
df_friends.select("name").show()

In [ ]:
df_friends.filter(df_friends.age > 60).show()

In [ ]:
df_friends.groupBy("age").count().orderBy("age", ascending=True).show()

In [ ]:
df_friends.groupBy("age").avg("num_friends").select("age", round("avg(num_friends)", 2).alias("avg_friends")).orderBy("avg_friends", ascending=False).show()

In [ ]:
avg_friends = spark.sql("SELECT age,ROUND(AVG(num_friends),2) AS avg_friends FROM friends GROUP BY age ORDER BY 2 DESC")
avg_friends.show()

## Book Dataset

In [78]:
from pyspark.sql import functions as func

df_book = spark.read.format("csv").option("header", "false").option("inferSchema", "true").load(f'{ml_100k_input_path}/Book')

# Split using a regular expression that extracts words
words = df_book.select(func.explode(func.split(df_book[0], "\\W+")).alias("word"))
wordsWithoutEmptyString = words.filter(words.word != "")

# Normalize everything to lowercase
lowercaseWords = wordsWithoutEmptyString.select(func.lower(wordsWithoutEmptyString.word).alias("word"))

# Count up the occurrences of each word
wordCounts = lowercaseWords.groupBy("word").count()

# Sort by counts
wordCountsSorted = wordCounts.sort("count" , ascending=False)

# Show the results.
wordCountsSorted.show(wordCountsSorted.count())

+----------------+-----+
|            word|count|
+----------------+-----+
|             you|  406|
|            your|  373|
|              to|  345|
|             the|  282|
|               a|  265|
|              of|  241|
|            that|  168|
|              is|  156|
|             for|  138|
|              in|  126|
|               i|  124|
|             are|  118|
|              if|  117|
|        business|  110|
|              it|  106|
|             and|   99|
|              on|   95|
|            this|   84|
|            have|   83|
|               s|   79|
|               t|   76|
|             can|   69|
|            with|   63|
|              at|   58|
|            will|   55|
|            what|   55|
|              my|   53|
|           about|   53|
|              be|   51|
|              as|   51|
|              do|   50|
|         product|   50|
|            they|   48|
|            time|   47|
|             how|   45|
|              re|   44|
|           there|   44|
